In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import requests
from pattern import web
from pattern.web import strip_between
from pattern.web import plaintext


import re

In [2]:
def get_list_of_dates_gender_posts(table_with_posts_html):
    date_list = []
    gender_list = []
    post_list = []
    
    tbl_rows = [row for row in posts_table.by_tag('tr') if row.by_tag('td')]
    
    for idx, row in enumerate(tbl_rows):
        if row.by_class("bold l pu"):
            date_and_gender = row.by_tag('span')
            full_date = ""
            gender = ""
            if len(date_and_gender) == 2:
                gender = plaintext(date_and_gender[0].content).strip().split()[0]
                date = plaintext(date_and_gender[1].content).split()
                if len(date)==4:
                    year = '2016'
                    day = date[-1]
                    month = month_dictionary[date[-2].lower()]
                    full_date = "-".join([str(day), str(month), year])
                elif len(date)==5:
                    year = date[-1]
                    day = date[-2].strip(",")
                    month = month_dictionary[date[-3].lower()]
                    full_date = "-".join([str(day), str(month), str(year)])
            elif len(date_and_gender) == 1:
                date = plaintext(date_and_gender[0].content).split()
                if len(date)==4:
                    year = '2016'
                    day = date[-1].strip(",")
                    month = month_dictionary[date[-2].lower()]
                    full_date = "-".join([str(day), str(month), year])
                elif len(date)==5:
                    year = date[-1]
                    day = date[-2].strip(",")
                    month = month_dictionary[date[-3].lower()]
                    full_date = "-".join([str(day), str(month), str(year)])
            date_list.append(full_date)
            gender_list.append(gender)
        elif row.by_class("l w pd"):
            content_in_div = row.by_tag('td')[0].by_class('narrow')[0].content
            new_html = strip_between('<blockquote>', '</blockquote>', content_in_div)
            new_html = (plaintext(new_html))
            new_string = new_html.replace("\n", " ")
            result = re.sub(r"http\S+", "", new_string)
            result = result.encode('utf-8', 'ignore')
            result = result.decode('ascii', 'ignore')
            post_list.append(result)
            
    return date_list, gender_list, post_list

In [3]:
#first check that the length of the date, gender, and post lists are the same
def write_list_to_string(dates, gender, posts):
    complete_string = ""
    for i in range(len(dates)):
        output = "|".join([dates[i], gender[i], posts[i]])
        complete_string = complete_string + output + "\n"
    return complete_string

In [4]:
month_dictionary = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9,\
                    "oct":10, "nov":11, "dec":12}

In [5]:
base_url = 'http://www.nairaland.com/{}/'

In [6]:
output_file = open('nairaland_data.txt', 'w')
output_file.write("Date | Gender | Post \n")

for i in range(4,100):
    url = base_url.format(str(i))
    try:
        #print url
        website_html = requests.get(url).text
        #print website_html
        dom = web.Element(website_html)
        
        all_tables_with_summary = [t for t in dom.by_tag('table') \
                                   if 'summary' in t.attributes and t.attributes['summary'] == "posts"]
        
        if len(all_tables_with_summary) == 0:
            print "no posts for this url ----->>"  + url
            continue
        else:
            #print "size of the summary table " + "---->>" + str(len(all_tables_with_summary))
            posts_table  = all_tables_with_summary[0]
            dates, gender, posts = get_list_of_dates_gender_posts(posts_table)
            
            #check that dates and posts are the same length
            if len(dates) == len(posts):
                print "size of dates and posts " + "---->>" + str(len(dates))
                #call function
                output_text = write_list_to_string(dates, gender, posts)
                #print output_text
                output_file.write(output_text)
            else:
                print "Uneven number of posts/dates for url"
                print url
            
            
    except:
        print url
        continue
output_file.close()

size of dates and posts ---->>32
no posts for this url ----->>http://www.nairaland.com/5/
size of dates and posts ---->>30
size of dates and posts ---->>32
size of dates and posts ---->>31
size of dates and posts ---->>25
no posts for this url ----->>http://www.nairaland.com/10/
no posts for this url ----->>http://www.nairaland.com/11/
no posts for this url ----->>http://www.nairaland.com/12/
no posts for this url ----->>http://www.nairaland.com/13/
size of dates and posts ---->>11
size of dates and posts ---->>32
size of dates and posts ---->>30
size of dates and posts ---->>32
size of dates and posts ---->>18
size of dates and posts ---->>31
size of dates and posts ---->>5
no posts for this url ----->>http://www.nairaland.com/21/
no posts for this url ----->>http://www.nairaland.com/22/
size of dates and posts ---->>32
size of dates and posts ---->>32
size of dates and posts ---->>32
size of dates and posts ---->>10
size of dates and posts ---->>32
size of dates and posts ---->>32
si